In [1]:
import pandas as pd
import numpy as np
import cv2

In [2]:
df1 = pd.read_csv('graph2_1.csv')
df2 = pd.read_csv('graph2_2.csv')
df3 = pd.read_csv('graph2_3.csv')
df4 = pd.read_csv('graph2_4.csv')

array1 = np.array(df1)
array2 = np.array(df2)
array3 = np.array(df3)
array4 = np.array(df4)

arrays = np.concatenate((array1,array2,array3,array4))

In [465]:
arrays = np.array(pd.read_csv('graph1.csv'))
arrays

array([[-9.50553161, -1.21391089],
       [-9.53723787, -1.19013478],
       [-9.60273342, -1.14289455],
       ...,
       [-9.52443655, -5.79051821],
       [-9.51097875, -5.73640197],
       [-9.50553161, -5.70895875]])

In [3]:
np.max(arrays, axis=0) - np.min(arrays, axis=0)

array([37.54260606, 52.60149074])

In [4]:
split_list = []
split_list.append(len(array1))
split_list.append((len(array1) + len(array2)))
split_list.append((len(array1) + len(array2) + len(array3)))
split_list

[136, 351, 1546]

In [5]:
array1.shape, array2.shape

((136, 2), (215, 2))

In [6]:
def normalization(array):
    return (array - np.min(array)) / (np.max(array) - np.min(array))

def standardization(array, size):
    width = np.max(array[:,0]) - np.min(array[:,0])
    height = np.max(array[:,1]) - np.min(array[:,1])
    rate = height / width
    array = normalization(array)
    array[:,0] = array[:,0] * size
    array[:,1] = array[:,1] * size * rate
    return array

standard_arrays = standardization(arrays, 500)

In [7]:
def make_graph(array):
    graph = np.zeros((int(array[:,0].max()) + 1,
                      int(array[:,1].max()) + 1), np.uint8)
    for each in array:
        graph[int(each[0]), int(each[1])] = 255
    return graph

graph = make_graph(standard_arrays)

In [8]:
def ligature_graph(graph, pos1, pos2):
    cv2.line(graph,
             (int(pos1[1]), int(pos1[0])),
             (int(pos2[1]), int(pos2[0])),
             255,
             1)

def ligature_graph_by_array(graph, array):
    for i in range(len(array) - 1):
        ligature_graph(graph, array[i], array[i+1])

ligature_graph_by_array(graph, standard_arrays[:split_list[0]])
ligature_graph_by_array(graph, standard_arrays[split_list[0]:split_list[1]])
ligature_graph_by_array(graph, standard_arrays[split_list[1]:split_list[2]])

In [8]:
ligature_graph_by_array(graph, standard_arrays)

In [9]:
cv2.imwrite('2.png', graph)

True

In [10]:
graph = cv2.imread('2.png', cv2.IMREAD_GRAYSCALE)

In [11]:
def get_profile_and_new_graph(graph):
    big_kernel = np.ones((10,10), dtype=np.uint8)
    tiny_kernel = np.ones((3,3), dtype=np.uint8)
    new_graph = cv2.erode(graph, big_kernel)
    eroded = cv2.erode(new_graph, tiny_kernel)
    profile = new_graph - eroded
    return profile, new_graph

In [26]:
import datetime
start = datetime.datetime.now()
profile_graph_list = []
new_graph = graph
c = 0
for i in range(20):
    profile, new_graph = get_profile_and_new_graph(new_graph)
    profile_graph_list.append(profile)
end = datetime.datetime.now()
end - start

datetime.timedelta(microseconds=21343)

In [12]:
c = 0
for i in range(1, 17):
    contours, _ = cv2.findContours(profile_graph_list[i], cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    length = 0
    for contour in contours:
        length += cv2.arcLength(contour, True)
    c += length / 2
c

0.0

In [486]:
profile_graph = profile_graph_list[0]
for i in range(1, len(profile_graph_list)-1):
    profile_graph += profile_graph_list[i]

cv2.imwrite('5.png', profile_graph)

True